# Clone from repo

# TESTs

## Create and join blocks

In [1]:
from src.base import Block, Boxtype
import src.vcs_function as eval
import src.box_plotter as bxplot

#test Blocks
b1 = Block(boxtype=Boxtype(1,2,4,True,True,True,1), rot='wlh')
b2 = Block(boxtype=Boxtype(1,2,3,True,True,True,1), rot='lwh')
b3 = Block(boxtype=Boxtype(1,1,2,True,True,True,1), rot='hwl')

#test join
print("b1:",b1); print("b2:",b2); print("b3:",b3)
t=b1.join(b2, 'y'); print(t,"b1+b2:",b1)
t=b1.join(b3, 'z'); print(t,"b1+b3:",b1)
t=b1.join(b1, 'x'); print(t,"b1+b1:",b1)

b1: Block: l: 4 w: 2 h: True weight: 1 volume: 8 occupied_volume: 8 items: {<src.base.Boxtype object at 0x0000020C0ABE31F0>: 1} ratio:1.0
b2: Block: l: 2 w: 3 h: True weight: 1 volume: 6 occupied_volume: 6 items: {<src.base.Boxtype object at 0x0000020C0ABE2020>: 1} ratio:1.0
b3: Block: l: True w: 2 h: 1 weight: 1 volume: 2 occupied_volume: 2 items: {<src.base.Boxtype object at 0x0000020C0ABE0D00>: 1} ratio:1.0
False b1+b2: Block: l: 4 w: 2 h: True weight: 1 volume: 8 occupied_volume: 8 items: {<src.base.Boxtype object at 0x0000020C0ABE31F0>: 1} ratio:1.0
False b1+b3: Block: l: 4 w: 2 h: True weight: 1 volume: 8 occupied_volume: 8 items: {<src.base.Boxtype object at 0x0000020C0ABE31F0>: 1} ratio:1.0
True b1+b1: Block: l: 8 w: 2 h: True weight: 2 volume: 16 occupied_volume: 16 items: {<src.base.Boxtype object at 0x0000020C0ABE31F0>: 2} ratio:1.0


## Plot container

In [2]:
def load_instance(filename = "instancia.txt", type="BF", id_instance=0):
    with open(filename, "r") as file:
        #discard first line
        file.readline()
      
        for j in range(100):
            file.readline()
            L, W, H = [int(x) for x in file.readline().split()]

            #read second line
            n = int(file.readline())
            #read n lines
            items = Itemdict()
            for i in range(n):
                id, l, rotx, w, roty, h, rotz, n = [int(x) for x in file.readline().split()]    
                item = Boxtype(id, l, w, h, rotx, roty, rotz, 1)
                items[item]=n
            
            if j == id_instance: return  items, L, W, H


In [3]:
def adyacent_blocks(cont,L,W,H,space,p):
    blocks = cont.aabbs
    last = blocks[-1]
    
    for block in blocks[:-1]:
        x_diff_max = max([block.xmin,last.xmin])
        x_diff_min = min([block.xmax,last.xmax])
        y_diff_max = max([block.ymin,last.ymin])
        y_diff_min = min([block.ymax,last.ymax])
        z_diff_max = max([block.zmin,last.zmin])
        z_diff_min = min([block.zmax,last.zmax])
        
        if abs(block.xmax - last.xmin) <= (last.xmax - last.xmin) * p and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.ymin < last.ymax and block.ymax > last.ymin )):

            block.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            block.covered_surface_face['X1']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface_face['X2']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)

        elif abs(block.xmin - last.xmax) <= (last.xmax - last.xmin) * p and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.ymin < last.ymax and block.ymax > last.ymin )):

            block.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            block.covered_surface_face['X2']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface_face['X1']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)

        elif abs(block.ymax - last.ymin) <= (last.ymax - last.ymin) * p and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):

            block.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            block.covered_surface_face['Y1']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface_face['Y2']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)

        elif abs(block.ymin - last.ymax) <= (last.ymax - last.ymin) * p and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):

            block.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            block.covered_surface_face['Y2']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface_face['Y1']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)

        elif block.xmin <= (block.xmax - block.xmin) * p:
            last.covered_surface+= ((last.ymax-last.ymin) * (last.zmax-last.zmin))
            last.covered_surface_face['X1']+= ((last.ymax-last.ymin) * (last.zmax-last.zmin))
        elif abs(block.xmax - space.l) <= (block.xmax - block.xmin) * p:
            last.covered_surface+= ((last.ymax-last.ymin) * (last.zmax-last.zmin))
            last.covered_surface_face['X2']+= ((last.ymax-last.ymin) * (last.zmax-last.zmin))
        elif block.ymin <= (block.ymax - block.ymin) * p:
            last.covered_surface+= ((last.xmax-last.xmin) * (last.zmax-last.zmin))
            last.covered_surface_face['Y1']+= ((last.xmax-last.xmin) * (last.zmax-last.zmin))
        elif abs(block.ymax - space.w) <= (block.ymax - block.ymin) * p:
            last.covered_surface+= ((last.xmax-last.xmin) * (last.zmax-last.zmin))
            last.covered_surface_face['Y2']+= ((last.xmax-last.xmin) * (last.zmax-last.zmin))

    print(type(cont))
    pass

In [4]:
def supported_blocks(blocks):
    X1 = 0; X2 = 0; Y1 = 0; Y2 = 0
    for block in blocks:
        if block.covered_surface_face['X1'] != 0:
            X1 += 1
        if block.covered_surface_face['X2'] != 0:
            X2 += 1
        if block.covered_surface_face['Y1'] != 0:
            Y1 += 1
        if block.covered_surface_face['Y2'] != 0:
            Y2 += 1
    X1/= len(blocks); X2/= len(blocks); Y1/= len(blocks); Y2/= len(blocks)
    print(f'{X1}  {X2}  {Y1}  {Y2} total: {(X1+X2+Y1+Y2)/4}')

# The main code

In [5]:

from src.base import Itemdict, BlockList, Aabb, Space

Space.filling = "bottom-up" # spaces are filled from bottom to top
Space.vertical_stability = True # boxes must be completly supported

items, L, W, H = load_instance(filename = "src/BR4.txt", type="BF", id_instance=1)

cont = Block(l=L,w=W,h=H)

# print("generating blocks...")
blocks = BlockList(items, type="general_blocks", cont=cont, min_fr=1.0, max_bl=1000) 

# greedy for single CLP
while True:
    space = cont.free_space.closest_space() #cuboide mas cercano a un vertice inferior del contenedor

    if space is not None:
      block = eval.eval_function(blocks,space,p=0.05,stored_blocks=cont.aabbs)
      # print("adding block...", block)
      
      cont.add_block(block, space)
      adyacent_blocks(cont,L,W,H,space,p=0.05)
      supported_blocks(cont.aabbs)
      items -= block.items
      blocks.remove_unconstructable(items)
      cont.free_space.filter(items)
    else:
        break

print(cont)


<class 'src.base.Block'>
0.0  0.0  0.0  0.0
<class 'src.base.Block'>
0.5  0.0  0.0  0.0
<class 'src.base.Block'>
0.6666666666666666  0.3333333333333333  0.0  0.0
<class 'src.base.Block'>
0.75  0.25  0.25  0.0
<class 'src.base.Block'>
0.8  0.2  0.4  0.0
<class 'src.base.Block'>
0.8333333333333334  0.16666666666666666  0.6666666666666666  0.16666666666666666
<class 'src.base.Block'>
1.0  0.2857142857142857  0.7142857142857143  0.14285714285714285
<class 'src.base.Block'>
1.0  0.25  0.875  0.375
<class 'src.base.Block'>
1.0  0.3333333333333333  0.8888888888888888  0.3333333333333333
<class 'src.base.Block'>
1.0  0.3  0.9  0.3
<class 'src.base.Block'>
1.0  0.36363636363636365  0.9090909090909091  0.36363636363636365
<class 'src.base.Block'>
1.0  0.3333333333333333  0.9166666666666666  0.3333333333333333
<class 'src.base.Block'>
1.0  0.3076923076923077  0.9230769230769231  0.38461538461538464
<class 'src.base.Block'>
1.0  0.2857142857142857  0.9285714285714286  0.35714285714285715
<class 's

# Plot the solution

In [6]:


box_dims = []
for aabb in cont.aabbs:
  box_dims.append([aabb.xmin,aabb.ymin,aabb.zmin,aabb.xmax,aabb.ymax,aabb.zmax])

bxplot.plot_container([L,W,H], box_dims)



# Push to repo